Получаем данные с coinmarketcap.com

In [50]:
import requests
import json

c_url = "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit=1500&sortBy=market_cap&sortType=desc&convert=USD,BTC,ETH&cryptoType=all&tagType=all&audited=false&aux=ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,max_supply,circulating_supply,total_supply,volume_7d,volume_30d,self_reported_circulating_supply,self_reported_market_cap"
c_resp = requests.get(cmc_url)
c_data = cmc_resp.json()["data"]["cryptoCurrencyList"]

Получаем данные с simpleswap

In [51]:
s_url = "https://simpleswap.io/api/v3/currencies?fixed=false&includeDisabled=false"
s_resp = requests.get(simpleswap_url)
s_data = ss_resp.json()

Собираем список монет simpleswap для сравнения с coinmarketcap

In [52]:
s_symbols = {coin['symbol'].upper() for coin in s_data}

Находим монеты из coinmarketcap, которых нет в simpleswap (volume24h - берем в usd)

In [53]:
missing_coins = []
for coin in c_data:
    symbol = coin["symbol"].upper()
    if symbol not in s_symbols:
        missing_coins.append({
            "name": coin["name"],
            "symbol": symbol,
            "volume24h": coin["quotes"][2]["volume24h"]
        })

Сортируем и смотрим на список

In [54]:
missing_coins_sorted = sorted(missing_coins, key=lambda x: x["volume24h"], reverse=True)

In [55]:
for i in missing_coins_sorted[0:3]:
    print(i)

{'name': 'First Digital USD', 'symbol': 'FDUSD', 'volume24h': 7412237054.568771}
{'name': 'BNB', 'symbol': 'BNB', 'volume24h': 3315500186.6801476}
{'name': '48 Club Token', 'symbol': 'KOGE', 'volume24h': 688219040.5896609}


Теперь переместим список монет в базу данных, где выполнял первое задание

In [57]:
import psycopg2

conn = psycopg2.connect(dbname="evercode_lab", user="postgres", password="Aa110604", host="localhost", port="5432")
cursor = conn.cursor()

cursor.execute("""
create table if not exists missing_coin (
    id serial primary key,
    name text,
    symbol text,
    volume24h numeric
)
""")
conn.commit()
# Очистим таблицу, чтоб не дублировать данные
cursor.execute("truncate table missing_coin")
conn.commit()

# Вставляем сами данные
for coin in missing_coins_sorted:
    cursor.execute(
        "insert into missing_coin (name, symbol, volume24h) values(%s, %s, %s)",
        (coin["name"], coin["symbol"], coin["volume24h"])
    )

conn.commit()

Проверим данные, выполнив запрос к бд и закроем соединение

Всего нашлось 527 монет

In [58]:
cursor.execute("select count(*) from missing_coin")
cnt = cursor.fetchone()[0]
print(cnt)

527


In [59]:
cursor.execute("select * from missing_coin limit 10")
rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.close()
conn.close()

(1582, 'First Digital USD', 'FDUSD', Decimal('7412237054.568771'))
(1583, 'BNB', 'BNB', Decimal('3315500186.6801476'))
(1584, '48 Club Token', 'KOGE', Decimal('688219040.5896609'))
(1585, 'Marina Protocol', 'BAY', Decimal('631972933.8468804'))
(1586, 'Nubila Network', 'NB', Decimal('387257443.8888502'))
(1587, 'Allora', 'ALLO', Decimal('250753001.27208203'))
(1588, 'Kite', 'KITE', Decimal('226884313.00203145'))
(1589, 'MetaArena', 'TIMI', Decimal('226863749.89064983'))
(1590, 'Momentum', 'MMT', Decimal('189662076.59569636'))
(1591, 'Mantle', 'MNT', Decimal('140420771.3777646'))
